![title](../images/logo_nao_digital.png)

# Tema 1: Conocimientos sobre despligue de ChatBots

## 1. Objetivo

Familiarizarse con los conceptos de Flask desplegar un ChatBot conversacional basado en un modelo de Deep Learning.

## Datos de la Medium Amira Rashid

Cómo se ha mencionado en el Anexo A, se ha provisto el archivo `conversations.json` que contiene los scripts de interacciones de usuarios y las respuestas que se espera que reciban, mismos que han sido provistos por el equipo de Amira a la empresa ChatBot Intelligence.

Esencialmente se trata de una estructura que contiene una serie de scripts de conversación:
* **tag:** es una etiqueta que clasifica el tipo de interacción entre el usuario y el Chatbot (por ejemplo, “saludo”, “despedida”, etcétera),
* **patterns:** conversaciones de ejemplo que se espera recibir como consultar al chatbot por parte de los usuarios,
* **responses:** es una lista de las respuestas en texto que se espera que los usuarios reciban.

## Arquitectura del ChatBot

Por otro lado, la arquitectura prototipada de interacción entre el usuario y el ChatBot es la siguiente:

![title](../images/chatbot_architecture.png)

En este sentido, el equipo teoriza que la herramienta debe funcionar como sigue:

* El usuario ingresa al chat en su dispositivo,
* El usuario introduce una consulta de texto,
* El texto se comunica a un servicio web donde se aloja el ChatBot,
* Este servicio procesa el texto y lo comunica a un modelo de Deep Learning para obtener una predicción de la respuesta a comunicar al usuario,
* La predicción es procesada y convertida a texto por el servicio del ChatBot,
* Esta respuesta se comunica al dispositivo del Chat

Este documento se desarrollarán scripts en Python para despleagar un ChatBot de conversación.

## 2. Librerias de trabajo

In [ ]:
# Instala libreria Pandas si no la tenemos
#!pip install numpy tensorflow pandas matplotlib seaborn wordcloud scikit-learn Flask

**Instalación de Spacy**

Spacy provee en su documentación las instrucciones para instalar su librería: https://spacy.io/usage

In [ ]:
#pip install -U pip setuptools wheel
#pip install -U spacy
#python -m spacy download en_core_web_sm
#python -m spacy download es_core_news_sm

In [ ]:
import os
import numpy as np
import spacy
import pandas as pd
import seaborn as sns
import json
import random
import pickle

import warnings
warnings.filterwarnings('ignore')

## 1. Elementos de Flask y HTML para un ChatBot

### 1.1 ¿Qué es Flask?

Cuando diseñamos sistemas que deben interactuar con el público a través de Internet, es necesario tener que adoptar algún framework de desarrollo web (en código) ya que éstos permiten crear una experiencia en línea que son fáciles de usar para las personas, por ejemplo a través de menús gráficos, botones, imágenes e indicaciones en texto. Los sitios y aplicaciones web bien diseñados son intuitivos, fáciles de usar y están optimizados para una variedad de dispositivos y navegadores, lo que garantiza que los usuarios puedan acceder y utilizar el servicio sin problemas.

Python tiene su propio framework de desarrollo web llamado **Flask** (https://flask.palletsprojects.com/en/2.2.x/). Éste es considerado un "micro-framework" ya que no requiere de herramientas o bibliotecas externas para funcionar y se enfoca en ofrecer sólo lo esencial para el desarrollo de sitio que se pueden optimizar para una variedad de dispositivos y navegadores. Entre las ventajas de usar Flask podemos mencionar:

* **Ligereza y simplicidad:** es liviano y no tiene muchas características y herramientas que otros frameworks pueden incluir por defecto. Esto hace que sea más fácil de entender y aprender,
* **Modularidad:** sus funciones se puede ampliar con extensiones, lo que lo hace muy flexible y adaptable a diferentes necesidades,
* **Documentación:** posee documentación detallada y útil que facilita el aprendizaje y la implementación de proyectos,
* **Buen rendimiento:** puede manejar grandes volúmenes de tráfico y solicitudes sin problemas,
* **Integración con otros servicios:**  se integra fácilmente con otras herramientas y servicios, lo que lo hace ideal para proyectos complejos que involucren varias tecnologías y plataformas.


### 1.2 ¿Cómo se implementa una interfaz para un servicio web?

En la construcción y diseño de interfaces para servicios web aparecen dos lenguajes muy famosos: HTML (Hypertext Markup Language) y CSS (Cascading Style Sheets).

Por un lado, HTML es el lenguaje de etiquetas que se utiliza para estructurar y organizar el contenido de una página, definir los elementos que la componen, por ejemplos sus títulos, párrafos, imágenes, enlaces, formularios y muchos otros. En adición, CSS se utiliza para definir la apariencia visual de una página web, como el color de fondo, el tipo de letra, la disposición de los elementos, los márgenes y otros aspectos visuales. Con CSS, los diseñadores pueden personalizar el diseño y la presentación de los sitios web, por ejemplo, a través de colores, bordes, transparencias o animaciones.

En la práctica el diseño de una interfaz de usuario, simple, funcional y atractivas para su uso involucra gran experiencia en diseño a través de HTML y CSS.

Para mayor referencia, ver:
    P. Acsany. (2022). HTML and CSS for Python Developers. Real Python. https://realpython.com/html-css-python/

**¿Cómo funciona HTML?**

Básicamente HTML usa una sintaxis basada en *etiquetas*. Estos son delimitadores que se utilizan para estructurar y dar formato a la información en una página web. Las etiquetas tienen la siguiente estructura en los archivos `.html`, que usualmente se emplean para escribir el código de un sitio web o aplicación

```
<etiqueta>
    ... algun texto ...
<\etiqueta>
```
**Nota:** El simbolo `<etiqueta>` abre la accion que ejecutaremos y `<\etiqueta>` la cierra. Dicha acción se ejecuta entre el texto delimitado por ambas.


Aquí hay algunas etiquetas HTML comunes:

* `<html>`: define el comienzo y el final de un documento HTML,
* `<head>`: contiene información sobre la página, como el título y los enlaces a hojas de estilo,
* `<body>`: delimita el contenido principal de la página.
* `<h1>` a `<h6>`:  se utilizan para definir los encabezados de la página, donde `<h1>` es el encabezado más importante y `<h6>` es el menos importante.
* `<p>`: define los párrafos en la página,
* `<a>`: crea enlaces a otras páginas web o a archivos dentro del sitio,
* `<img>`: permite insertar imágenes,
* `<table>`: nos deja crear tablas en la página.

**Pregunta:**
* Localiza el archivo *example_1.html* que se te ha provisto. Este tiene el siguiente código:
```
!DOCTYPE html>
<html>
  <!-- Este es un comentario -->
  <head>
    <title>Ejemplo 1 de HTML</title>
  </head>
  <!-- Definimos el cuerpo del documento-->
  <body>
    <!-- Este es un encabezado -->
    <h1>Ejemplo 1: Chat de conversación: </h1>
    <!-- Parrafos de conversacion -->
    <p>Usuario: Hola, ¿quien eres?.</p>
    <p>ChatBot: Soy un Bot conversacional! Mucho gusto :)</p>
  </body>
</html>
```

  * ¿Cómo se ve el archivo si lo abres con un editor de texto?
  * ¿Cómo se ve el archivo si lo abres con tu navegador (por ejemplo, safari, chorme, firefox, etcétera)?



### 1.3 ¿Cómo se relaciona Flask con los modelos de Aprendizaje de Máquina?

Como hemos dicho, a través de Flask se puede diseñar una interfaz de un servicio web, con la que los usuarios interactuarán con una aplicación. Sin embargo, una ves que hemos entrenamos un modelo de aprendizaje de máquina, lo más relevante es comunicar las predicciones para que sean usadas en el problema de negocio que se busca resolver. Cuando las predicciones no se van a usar de forma inmediata, usualmente se deben guardar en una base de datos, un servidor ftp o en un gestor de archivos en la nube (como S3 de Amazon Web Services); sin embargo, si se requieren usar de manera manera continua o intensiva, es necesario implementar algún servicio con el que los usuarios puedan interactuar directamente para obtener predicciones específicas con ciertos parámetros (por ejemplo, si tenemos un modelo que predice el tiempo de llegada de un servicio de transporte privado, el teléfono deberá comunicar los datos del destino al modelo y este le deberá regresar una estimación de cuanto tardará su servicio, ¡para cada usuario que use nuestra aplicación en tiempo real!).

El ChatBot es un caso de negocio de éste estilo, ya que un usuario enviará mensajes de manera continua mediante una interfaz y el modelo de aprendizaje automático deberá ayudarnos a predecir la respuestas que más se adecue el texto recibido.

Flask permite implementar ésta clase de servicios de intermediación entre interfaces gráficas y modelos entrenados de aprendizaje de máquina, con un servicio llamado API REST. Éste no es más que una interfaz de programación de aplicaciones, que permite interactuar con una computadora o un sistema
entienda una cierta solicitud, a través de deteminados parámetros y como resultados nos devuelva un conjunto de datos o ejecute una función.

Para obtener una predicción básicamente una API REST debe: 1) recibir la información del individuo sobre el que queremos hacer una predicción, posteriormente 2) transformar estos datos de una forma entendible para el modelo, 3) con dicha información se aplicar el modelo entrenado para obtener las predicciones y 4) comunicar el resultado de la predicción a un servicio web. Dicho proceso se ejemplifica a continuación:

**Figura 1:** *Esquema de una API REST en Flask, actuando como intermediario entre un servicio web, accedido por computadora y un modelo que permite obtener predicciones.*
![title](../images/flask_api_model.png)


Típicamente, las API REST se asocian con *recursos*, los cuales son tipos de objetos, datos o servicios al que se puede acceder con dicho sistema a través de un cliente. Las acciones que se pueden ejecutar con una API REST reciben el nombre de verbos las que podemos listar las siguientes:

* **GET:** Para obtener un recurso o colección de datos.
* **POST:** Para crear un recurso (y/o añadirlo a una colección).
* **PUT:** Para modificar un recurso.
* **DELETE:** Para eliminar un recurso.

En nuestro contexto, si queremos obtener la predicción de un modelo, podemos ejecutar una acción de tipo **GET** a la cual le comunicaremos los detalles del individuo de interés y como resultado la API REST debe entregar la variable que el modelo predice para éste.



### 1.4 ¿Cómo se puede desplegar un ChatBot usando Flask?

Una vez entrenado un modelo conversacional de aprendizaje de máquina, para desplegar un ChatBot, básicamente tenemos que juntar todo lo que hemos discutir hasta aquí como una serie de pasos:

* A. Crear una interfaz web con HTML (y CSS para un diseño amigable), donde el usuario enviará y recibirá mensajes de texto con el Bot.

* B. Construir una aplicación con Flask, que:
    * B.1 Renderize la interfaz anterior, exponiéndola en un servicio en una computadora. Típicamente este se realiza en un puerto específico (como el puerto 5000),
    * B.2 También funcione como una API REST para el modelo, de forma en que el mensaje que un usuario envia se procese y se comunicado al modelo, obtienen la predicción correspondiente y exponiendo el mensaje respuesta en la interfaz.

En la siguiente sección veremos ejemplos de como implementar aplicaciones de Flask, hasta implementar un ejemplo de un ChatBot, bajo la arquitectura recién descrita.

## 2. Usando Flask para construir un ChatBot

### 2.1 Primera aplicación:

Para construir una aplicación Flask, se debe crear un script en Python, instanciando un `app`
con la clase `Flask`. El siguiente es un código de ejemplo del script llamado `app_example_1.py`
y definiendo las rutas que expondrá nuestro servicio web como sigue:

```
# Ejemplo 1 de una aplicacion en Flask
from flask import Flask

# Instancia el app
app = Flask(__name__)

# Ruta raiz
@app.route('/')
def hello_world():
    return 'Hola Mundo! Soy tu primer ejemplo ;)'

if __name__ == '__main__':
    app.run()
```

Para inicializar esta aplicación, en una terminal debemos correr el comando `python mi_script.py`, es decir, en el ejemplo:

```
python app_example_1.py
```
**Notas:** En la terminal, se debe ejecutar la versión de Python que tenga instalado todos los paquetes de trabajo, particularmente Flask.

Ello expondrá la aplicación en la dirección `http://127.0.0.1:5000/`. Para verla, simplementa abrimos esa liga con cualquier navegador web de nuestra máquina local, y ahora se debe ver un mensaje que dice "Hola Mundo! Soy tu primer ejemplo ;)"

Para cerrar el programa basta ejecutar la combinación del teclado: `tecla Ctlr + C`.




### 2.2 Segunda aplicación: renderizamos un archivo HTML

Como hemos dicho Flask también nos deja exponer servicios web que se han diseñado en HTML, basicamente esto significa que si creamos un archivo de éste tipo, podemos hacerlo accesible a los usuarios que acceden a los puertos donde Flask ha alojado el servicio correspondiente.

En la parte inferior se muesta el código de ejemplo `app_example_2.py` para una aplicación que
renderiza al archivo `example_1.html`. Asegúrate que dicho archivo se encuentre en la misma carpeta que el archivo .py o el código marcará errores!:

```
"""
Ejemplo 2 de una aplicacion en Flask
Renderizando un archivo de HTML
"""

# Importamos "render_template"
from flask import Flask, render_template

# Instancia el app
# Nota: Cuida que example_1.html este
# en el mismo directorio donde esta este archivo!
app = Flask(__name__, template_folder='.')

# Definemos la ruta para
@app.route("/")
def render():
    # Indicamos que la ruta de regresar el template
    # renderizado del archivo "example_1.html"
    return render_template("example_1.html")

# Iniciamos la aplicación
if __name__ == '__main__':
    app.run()
```

Nuevamente, para ejecutarlo bastára correr el código `python app_example_2.py` y se podrá acceder al servicio en la dirección `http://127.0.0.1:5000/` desde cualquier navegador de nuestra computadora. El servicio se puede concluir ejecutando la combinación  de teclado `tecla CTRL + c`.

### 2.3 Creando un API para desplegar un modelo

Ahora veremos un ejemplo de como se puede usar Flask para desplegar un modelo ya entrenado y obtener predicciones.

En este caso, vamos a entrenar una regresión linear con la librería Sklearn, que predice si una persona vivio tras un año de enfermase de diabetes en sujetos de una población (véase https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset):

**train_example_3.py**
```
"""
Entrena una regresion lineal sobre el conjunto
diabetes de Skleanr
"""

import pickle
import numpy as np

# Sklearn
from sklearn import linear_model
from sklearn import datasets

# Conjunto de datos diabetes
diabetes = datasets.load_diabetes()

# Entrenamos el modelo de regresion lineal con
# una sola variable
X = diabetes.data[:, np.newaxis, 2]
model = linear_model.LinearRegression()
model.fit(X, diabetes.target)

# Salvamos el modelo entrenado con formato pickle
pickle.dump(model, open('model_diabetes.pkl', 'wb'))
```

Ejecutemoslo con `python train_example_3.py`, como resultado este creara el archivo `model_diabetes.pkl` que es la versión comprimida del modelo entrenado.

**app_example_3.py**

El siguiente script crea la aplicación que crea el API REST para crear predicciones a partir del parámetro feature:

```
"""
Aplicacion que crea un API REST para obtener
las prediccion de una regresion lineal
previamente entrenada
"""

# Flask
from flask import Flask, request
import pickle
import os

# --------- Aplicacion en Flask ------------------- #
# Instancia la app
app = Flask(__name__, template_folder='.')

# Define una ruta para saber si la aplicacion
# esta corriendo
@app.route('/healtCheck')
def index():
    return "true"

# Ruta con accion get para obtener la predicion
@app.route('/prediction/', methods=['GET'])
def get_prediction():
    # Convierte el parametro f recibido por GET a
    # numero flotante
    feature = float(request.args.get('feature'))

    # Cargamos el modelo ya entrenado
    CURRENT_PATH = app.static_folder.replace('static', '')
    model_path = os.path.join(CURRENT_PATH, 'model_diabetes.pkl')
    model = pickle.load(open(model_path, 'rb'))
    # Obtiene la prediccion para el valor recibido
    pred = model.predict([[feature]])

    # Comunimos a la ruta el valor de la prediccion
    return str(pred)


if __name__ == '__main__':
    app.run()

```

La aplicación consta de dos partes: 1) Crea una ruta `/healtCheck` que nos permite saber si la app está activa devolviendo el texto `true` a visitar la dirección `http://127.0.0.1:5000/healtCheck` en un navegador, y 2) nos permite interacturar con una aplicación API REST, esencialmente habilitando que la acción GET se pueda habilitar sobre la ruta `http://127.0.0.1:5000/prediction/` para recibir un parámetro numérico.

Para crear predicciones, basta ingresar a la dirección `http://127.0.0.1:5000/prediction/?feature=numero_real` donde `numero_real` es el valor numérico sobre el que vamos a correr la predicción.

A continuación se muestran imagenes del proceso:

**Figura 2:** *Ruta para verificar que la aplicación funcion (Healt Check)*

![title](../images/example_3_healtcheck.png)


**Figura 3:** *Predicción para el valor feature=11*

![title](../images/example_3_prediction.png)

Finalmente, otra forma de interactuar con el API es emplear el comando `curl` de Unix mientra la app está activa:

```
# usando una terminal tipo Unix
curl http://127.0.0.1:5000/prediction/?feature=11
```

**Figura 4:** *Predicción para el valor feature=11 usando el comando curl*

![title](../images/example_3_curl.png)


### 2.4 Ejemplo 4: ChatBot con Flask

Por último, ahora revisaremos la implementación de un ChatBot usando Flask (**app_example_4.py**), para ello partiremos del modelo que fue entrenado en el notebook del Tema 2, correspondiente al archivo de conversaciones `sample.json`,y de los archivos pickle que generamos en el preprocesamiento de los datos, así como el modelo entrenado. Concretamente nos referimos a los siguientes:

* `category_answers.pkl`: corresponde al un diccionario que conjunta las categorías asociadas a las preguntas `sample.json` y las respuestas que cada usuario puede recibir.
* `sample_vectorizer_bow.pkl`: es el transformador de Sklearn que permite representar un texto limpio en la bolsa de palabras con la que se entrenó el modelo.
* `sample_categories.pkl`: es una lista que contiene el nombre de las categorías de las preguntas de los usuarios, procedentes del archivo `sample.json`. Dicha lista se ordenó de acuerdo al orden en que se representaron las columnas en la representación del One Hot Encoding con la función `.get_dummies` de Pandas.
* `sample_model.pkl`: es la versión comprimida del modelo que se entrenó en Tensorflow para predecir a que categoría de respuesta pertenece un mensaje de texto, usando su representación en una bolsa de palabras.

La aplicación Flask emplea un archivo HTML denominado `index.html` el cual define la estructura del ChatBot y el menú con el que el usuario le enviará mensajes. Cabe destacar que la forma de intercambiar información entre la aplicación Flask y el servicio web que renderizará `index.html` toma los siguientes principios:


1) La aplicación implementa dos rutas: `/healtCheck` que funciona simplementa como un check de salud para saber si la aplicación y una ruta raiz `/` donde se alojará la interfaz del Chatbot.
2) En la ruta raíz se pueden realizar acciones GET y POST. En la acción GET, la ruta devolverá el template `index.html` renderizado. En complemento, en la acción POST, si el usuario ha introducido una pregunta, lo que se hará es transformar el texto y comunicarselo al modelo para obtener la respuesta correspondiente. La pregunta del usuario y la correspondiente respuesta se guardarán en una lista que será comunicada al cuerpo del HTML para se insertada en un formato de párrafos `<p> ... </p>`, con lo cual conseguiremos que esta conversación se muestre en la interfaz del ChatBot.

Sobre este último punto, cabe destacar que la implementación aprovecha una característica interesante que tiene Flask; éste permite tomar variables del código Python y renderizarlas dentro del código HTML con Jijna2 (https://flask.palletsprojects.com/en/2.2.x/templating/). Concretamente, nos referimos a la sección de código:

```
{% for c in chat %}
    <p>{{c}}</p>
{% endfor %}
```

En la aplicación Flask, sucede que transferimos información al archivo HTML usando la función `render_template('index.html', chat=conversations)`, como se aprecia la variable chat del HTML se alimenta de la lista `conversations` en donde guardamos la pregunta del usuario y la respuesta del ChatBot en texto.

El código de la aplicación `app_example_4.py` es el siguiente:

```
"""
Aplicacion que crea un API REST para obtener
las prediccion de una regresion lineal
previamente entrenada
"""
#
import numpy as np
import spacy
import random
import pickle

# Flask
from flask import Flask, request, render_template
import pickle

# Cargamos el modelo de lenguaje en español de Spacy
nlp = spacy.load('es_core_news_sm')

# ---- archivos del procesamiento del modelo ----- #

# diccionario de respuestas del bot por categoria
sample_category_answers = pickle.load(
    open('notebooks/category_answers.pkl','rb')
    )

# Transformador para texto a nube de palabras
sample_vectorizer_bow = pickle.load(
    open('notebooks/sample_vectorizer_bow.pkl','rb')
    )

# Lista ordenada de las categorias de los tags a predecir
sample_categories = pickle.load(
    open('notebooks/sample_categories.pkl','rb')
    )

# Modelo previamente entrenado
model = pickle.load(
   open('notebooks/sample_model.pkl','rb')
    )

# --------- Funciones auxiliares --------------- #


def text_pre_process(message: str):
    """
    Procesa el texto del nuevo mensaje
    """
    # Procesa el mensaje con spaCy
    tokens = nlp(message)
    # remueve signos de puntuacion y lematiza
    new_tokens = [t.orth_ for t in tokens if not t.is_punct]

    # pasa a minusculas
    new_tokens = [t.lower() for t in new_tokens]

    # une los tokens procesados con un espacio
    clean_message = ' '.join(new_tokens)

    return clean_message


def bow_representation(message: str) -> np.array:
    """
    Obtiene la representacion del mensaje en su
    codificacion de la nube de palabras
    """
    bow_message = sample_vectorizer_bow.transform(
        [message]
        ).toarray()

    return bow_message


def get_prediction(
        bow_message: np.array
        ) -> int:
    """
    Obtiene la prediccion de la categoria
    que corresponde al mensaje
    """

    # Calcula el indice entero al que corresponde la catforia
    prediction = model(bow_message).numpy()

    # Obtiene el indice de la entrada con probabilidad mayor
    index = np.argmax(prediction)

    predicted_category = sample_categories[index]

    return predicted_category


def get_answer(category: str) -> str:
    """
    Obtiene el mensaje de respuesta para una categoria
    """
    # Obtiene las respuestas de la categoria
    answers = sample_category_answers[category]

    # Selecciona una respuesta al azar
    ans = random.choice(answers)

    return ans

# --------- Aplicacion en Flask ------------------- #
# Instancia la app
app = Flask(__name__, template_folder='.')

# Define una ruta para saber si la aplicacion
# esta corriendo
@app.route('/healtCheck')
def index():
    return "true"

# Define la interaccion entre la persona y el ChatBot
# a partir de sitio web del archivo index.html

# lista para guardar conversaciones
conversations = []


@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'GET':
        return render_template('index.html')
    if request.form['question']:
        # Procesa la pregunta para obtener la prediccion
        # del modelo
        raw_question = request.form['question']
        clean_question = text_pre_process(raw_question)
        bow_question = bow_representation(clean_question)
        prediction = get_prediction(bow_question)
        bot_answer = get_answer(prediction)

        # Crea textos de respuesta y pregunta
        question = 'Usuario: ' + raw_question
        answer = 'ChatBot: ' + bot_answer

        # Guarda los textos de conversacion en la lista
        conversations.append(question)
        conversations.append(answer)

        # comunica los textos de conversacion al archivo html
        return render_template('index.html', chat=conversations)
    else:
        return render_template('index.html')


if __name__ == '__main__':
    app.run()

```

Como sabemos, el despliegue de la aplicación se logra con el comando:

```
python app_example_5.py
```

**Notas:**
* Antes de correr ésta app, asegurarse que en el mismo directorio de trabajo se encuentren los archivos `category_answers.pkl`, `sample_vectorizer_bow.pkl`, `sample_categories.pkl` y `sample_categories.pkl`.



En la siguiente imagen se muestra la interfaz del Bot y una conversación sostenida con éste:

**Figura 5:** *ChatBot implementado con el modelo entrenado en el tema 2*

![title](../images/example_4_chatbot.png)

### 6. Entregables

A. Modifica el código presentado anteriormente para crear un ChatBot con el contenido del archivo **conversations.json** provisto por el equipo de Amira Rashid. La estructura de carpetas que se sugiere seguir para construir el ChatBot es la siguiente:

```
# Estructura de carpeta del ChatBot de la Medium Amira Rashid

app.py # Archivo de la aplicacion de Flask
index.html # Archivo HTML de la interfaz del ChatBot
data/ # Folder con datos de procesamiento del modelo en formato pickle
    archivo_1.pkl
    archivo_2.pkl
    ...
    archivo_n.pkl
```

Se sugiere agregar textos o diseños en el Archivo `index.html` para mejorar la presentación del ChatBot ante el equipo de la influencer Amira.

El proyecto anterior debe ser proporciona en formato .zip con nombre `chatbot_amira.zip`

B. Adicionalmente, deberás grabar un video en formato `.mp4` donde se observe:
    b.1 El ChatBot desplegado en un navegador de tu computadora local,
    b.2 La interacción con el ChatBot que implementaste para las siguientes acciones:
        I) Saludar al bot,
        II) Preguntarle su nombre,
        III) Obtener los vaticiones del Bot de al menos 3 signos zodiacales,
        IV) Despedida del bot.